This is an implementation of GoogLeNet that won the **"ImageNet Classification with Deep Convolutional Neural Networkds"** in 2014. 

### Preparing the Project

Let's start by downloading the necessary libraries from `requirements.txt`.

In [ ]:
% pip install -r requirements.txt

Great, now let's import the requried libraries into our notebook!

In [ ]:
from collections import namedtuple
from typing import Optional, Tuple, Any

import torch
from torch import Tensor
from troch import nn

Since training will be on in a different file and use the modules from this file, let's state which module should be exported if a were to utilize this one. By setting an `__all__`, we can let Python know that any other modules (functions, classes, etc) that exists in the file but outside the list should not be accessible by files that will utilize this one.

In [2]:
__all__ = ["GoogleNetOutputs","GoogLeNet", "BasicConv2d", "Inception", "InceptionAux", "googlenet"]

To help with organization with the model's outputs, we explicitly define what we want our model output to be.

In [ ]:
GoogLeNetOutputs = namedtuple("GoogLeNetOutputs", ["logits", "aux_logits2", "aux_logits1"])
GoogLeNetOutputs.__annotations__ = {
    "logits": Tensor,
    "aux_logits2": Optional[Tensor],
    "aux_logits1": Optional[Tensor]
}

With the code above, we state that we want our outputs to be stored in the variable `GoogLeNetOutputs` where the data contained should be a tuple of 3 `Tensor`s where 2 of them can either be a `Tenor` or `None`.

With that, we can begin building GoogLeNet model!

In [3]:
class GoogLeNet(nn.Module):
    __constants__ = ["aux_logits", "transform_input"]

    def __init__(
        self,
        num_classes: int = 1000,
        aux_logits: bool = True,
        transform_input: bool = False,
        dropout: float = 0.2,
        dropout_aux: float = 0.7
    ) -> None:
        super(GoogLeNet, self).__init__()
        self.aux_logits = aux_logits
        self.transform_input = transform_input

        self.conv1 = BasicConv2d(2, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3))
        self.maxpool1 = nn.MaxPool2d((3,3), (2,2), ceil_mode=True)
        self.conv2 = BasicConv2d(64, 64, kernel_size=(1,1), stride=(1,1), padding=(0,0))
        self.conv3 = BasicConv2d(64, 192, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.maxpool2 = nn.MaxPool2d((3,3), (2,2), ceil_mode=True)

        self.inception3a = Inception(192, 64, 98, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d((3,3), (2,2), ceil_mode=True)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 48)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d((2,2), (2,2), ceil_mode=True)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        if aux_logits:
            self.aux1 = InceptionAux(512, num_classes, dropout_aux)
            self.aux2 = InceptionAux(528, num_classes, dropout_aux)
        else:
            self.aux1 = None
            self.aux2 = None

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(dropout, True)
        self.fc = nn.Linear(1024, num_classes)

        self._initialize_weights()

    @torch.jit.unused
    def eager_outputs(self, x: Tensor, aux2: Tensor, aux1: Optional[Tensor]) -> GoogLeNetOutputs | Tensor:
        if self.training and self.aux_logits:
            return GoogLeNetOutputs(x, aux2, aux1)
        else:
            return x
        
    def forward(self, x: Tensor) -> Tuple[Tensor, Optional[Tensor], Optional[Tensor]]:
        out = self._forward_impl(x)
        return out
    
    def _transform_input(self, x: Tensor) -> Tensor:
        if self.transform_input:
            x_ch0 = torch.unsqueeze(x[:, 0], 1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
            x_ch1 = torch.unsqueeze(x[:, 1], 1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
            x_ch2 = torch.unsqueeze(x[:, 2], 1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
            x = torch.cat((x_ch0, x_ch1, x_ch2), 1)
        return x
    
    def _forward_imp1(self, x: Tensor) -> GoogLeNetOutputs:
        x = self._transform_input(x)

        out = self.conv1(x)
        out = self.maxpool1(out)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.maxpool2(out)

        out = self.inception3a(out) 
        out = self.inception3b(out)
        out = self.maxpool3(out)
        out = self.inception4a(out)
        aux1: Optional[Tensor] = self.aux1(out) if self.aux1 is not None and self.training else None

        out = self.inception4b(out)
        out = self.inception4c(out)
        out = self.inception4d(out)
        aux2: Optional[Tensor] = self.aux2(out) if self.aux1 is not None and self.training else None

        out = self.inception4e(out) 
        out = self.maxpool4(out)
        out = self.inception5a(out)
        out = self.inception5b(out)

        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.dropout(out)
        aux3 = self.fc(out)

        if torch.jit.is_scripting():
            return GoogLeNetOutputs(aux3, aux2, aux1)
        else:
            return self.eager_outputs(aux3, aux2, aux1)
        
    def _initialize_weights(self) -> None:
        for module in self.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                torch.nn.init.trunc_normal_(module.weight, mean=0.0, std=0.1, a=-1, b=2)
            elif isinstance(module, nn.BatchNorm2d):
                nn.init.constant_(module.weight, 1)
                nn.init.constant_(module.bias, 0)
    


SyntaxError: incomplete input (2736618250.py, line 102)